In [ ]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

In [ ]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

In [ ]:
fieldnames(typeof(plasma_eq.rzphi))

### CAlc. Metric

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric
metric_result = nothing
try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
 
catch e
    println("❌Error: $e")
end

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [ ]:
using Revise

In [ ]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [ ]:
# Fourfit Matrix Calculation - Complete Example
println("🚀 Fourfit Matrix Calculation")
println("="^40)

using .FourfitMetric
matrix_result = nothing
try
    # Step 1: Matrix calculation with correct arguments
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,       # metric tensor data
        plasma_eq.sq,        # 1D profile spline
        plasma_eq.rzphi,     # 2D geometry spline
        1.367;               # psio (toroidal flux normalization)
        nn=1,                # toroidal mode number
        mlow=-4,             # minimum poloidal mode
        mhigh=4,             # maximum poloidal mode
        power_flag=false,
        feval_flag=false,
        sas_flag=true,
        verbose=true
    )
    
    # Step 2: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix storage sizes:")
    println("      F-matrix: $(size(matrix_result.fmats))")
    println("      G-matrix: $(size(matrix_result.gmats))")
    println("      K-matrix: $(size(matrix_result.kmats))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    println("✅ Matrix calculation completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("🔍 Error type: $(typeof(e))")
end

### Redaing imats(original DCON output)

In [ ]:
using XLSX

In [ ]:
# ────────────────────────────────────────────────────────────────
# Internal constants - Excel column positions (1-based)
const _COLS = Dict(
    :A => (4, 5),
    :B => (6, 7),
    :C => (8, 9),
    :D => (10, 11),
    :E => (12, 13),
    :H => (14, 15),
)
# ────────────────────────────────────────────────────────────────
mutable struct ImatsReader
    path      :: String
    psi       :: Vector{Float64}
    m_values  :: Vector{Int}
    npsi      :: Int
    mpert     :: Int
    A         :: Array{ComplexF64,3}
    B         :: Array{ComplexF64,3}
    C         :: Array{ComplexF64,3}
    D         :: Array{ComplexF64,3}
    E         :: Array{ComplexF64,3}
    H         :: Array{ComplexF64,3}
end

function ImatsReader(path::AbstractString)
    isfile(path) || throw(ArgumentError("File not found: $path"))
    # Temporary storage for variables
    temp_storage = []
    # 1) Open Excel file and read all data rows temporarily
    XLSX.openxlsx(path) do xf
        sheet = xf[1]
        # Use XLSX.eachtablerow to iterate through rows
        for row in XLSX.eachtablerow(sheet)
            # Skip if first column value is not a number (header or empty row)
            ψ = row[1]
            isa(ψ, Number) || continue
            # Read mi, mj and other values from the row
            mi = Int(row[2])
            mj = Int(row[3])
            # Store entire row data (columns containing real/imaginary parts of A~H)
            # Read data up to 15th column
            row_data = [row[c] for c in 1:15]
            push!(temp_storage, (ψ, mi, mj, row_data))
        end
    end # End of XLSX.openxlsx do block
    isempty(temp_storage) && error("No numeric data rows found in $path")
    
    # 2) Organize ψ / m lists
    psi_vals = sort(unique(getindex.(temp_storage, 1)))
    m_vals_all = vcat(getindex.(temp_storage, 2), getindex.(temp_storage, 3))
    m_vals   = sort(unique(m_vals_all))
    nψ       = length(psi_vals)
    mpert    = length(m_vals)
    
    # Create dictionaries for value -> index conversion
    ψidx = Dict(v => i for (i, v) in enumerate(psi_vals))
    midx = Dict(v => i for (i, v) in enumerate(m_vals))
    
    # 3) Initialize empty arrays to store final matrices
    shp = (nψ, mpert, mpert)
    mats = Dict(name => zeros(ComplexF64, shp) for name in keys(_COLS))
    
    # 4) Fill arrays with temporarily stored data
    for (ψ, mi, mj, row_cells) in temp_storage
        k = ψidx[ψ]
        i = midx[mi]
        j = midx[mj]
        for (name, (recol, imcol)) in _COLS
            # row_cells has 1-based indexing
            real_val = row_cells[recol]
            imag_val = row_cells[imcol]
            # Handle empty cells (nothing) or non-numeric values as 0.0
            re = isa(real_val, Number) ? Float64(real_val) : 0.0
            im = isa(imag_val, Number) ? Float64(imag_val) : 0.0
            mats[name][k, i, j] = ComplexF64(re, im)
        end
    end
    
    # Create and return final object
    return ImatsReader(String(path), psi_vals, m_vals, nψ, mpert,
               mats[:A], mats[:B], mats[:C], mats[:D], mats[:E], mats[:H])
end

# ────────────────────────────────────────────────────────────────
# Internal: ψ index interpretation
function _ψindex(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                   ψ_value::Union{Nothing,Real}=nothing)
    ψ_index !== nothing && return ψ_index
    ψ_value !== nothing || error("Either ψ_index or ψ_value must be specified")
    return argmin(abs.(r.psi .- ψ_value))
end

# ────────────────────────────────────────────────────────────────
"Return single matrix: name ∈ (:A,:B,:C,:D,:E,:H)"
function get_matrix(r::ImatsReader, name::Symbol;
                    ψ_index::Union{Nothing,Int}=nothing,
                    ψ_value::Union{Nothing,Real}=nothing)
    name = Symbol(uppercase(String(name)))
    hasproperty(r, name) || error("unknown matrix $name")
    k = _ψindex(r; ψ_index, ψ_value)
    return getfield(r, name)[k, :, :]
end

"Return one ψ cross-section (6 matrices) as Dict"
function get_matrices(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                         ψ_value::Union{Nothing,Real}=nothing)
    k = _ψindex(r; ψ_index, ψ_value)
    return Dict(name => getfield(r, name)[k, :, :] for name in (:A,:B,:C,:D,:E,:H))
end

In [ ]:
reader = ImatsReader("../src/DCON/validate/imats.xlsx")


In [ ]:
reshape(matrix_result.amats[129, :], 9, 9)./reader.A[end,:,:]